In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Challenge: try best not use any AI for this. It gives a nice emotional kick!

In [54]:
# I want to generate sequences based on my own transition prob matrix to test

transition_matrix = np.zeros((4, 4))
#order here is A,C,G,T. Each row corresponds to the previous letter and each col to the following letter.

transition_matrix = np.array([[0.25, 0.25, 0.25, 0.25],
                              [0.05, 0.05, 0.80, 0.10], #Cs are likely followed by G
                              [0.80, 0.05, 0.05, 0.10], # Gs are likely followed by A  
                              [0.25, 0.25, 0.25, 0.25]])
Alphabets_of_DNA_seq_dict = {0:"A",1:"C",2:"G",3:"T"}
Alphabets_of_DNA_seq_list = ["A","C","G","T"]
start_prob = np.array([0.25,0.25,0.25,0.25])

def gen_seq(tm,n,start_prob):
    generated_seq = ""
    generated_seq += np.random.choice(Alphabets_of_DNA_seq_list,p=start_prob)
    for i in range(n-1):
        last_letter = generated_seq[-1]
        for key, value in Alphabets_of_DNA_seq_dict.items(): 
            if value == last_letter:
                generated_seq += np.random.choice(Alphabets_of_DNA_seq_list, p = tm[key])
                #print(key)
        #print(i)
    return generated_seq

In [55]:

seq_of_interest = gen_seq(transition_matrix,200,start_prob)
print(seq_of_interest)
len(seq_of_interest)

GACGAACGATCGACGCGTTACGCGAACGAATCGAGACGATAACGATCGAGAATATCGATTAAGACGAGACGACGAAACGACGACGAGACGATTGACGAGAGAGAGATATCGATTTTCGTCGAGACCGACGACGATCGTAACGATTAGAGATACGCGTCGATGATGATTTTTATCCGAATATCGATGCGAGCGATTCGACG


200

In [72]:
estimated_transition_matrix = np.zeros((4, 4))
Alphabets_of_DNA_seq_dict_with_alpha_as_the_key = {"A": 0, "C": 1, "G": 2, "T": 3}

order_of_markov = 1
viewing_window = order_of_markov + 1
Number_of_windows = len(seq_of_interest)- order_of_markov

'''
Loop through all of the windows, 
 In each window, take the letters of length order + 1.
 Check if this seq of letters exists in the dict. If yes, add one to value.
 If add a new key and add one. 

NOw here is the challenging bit. SHould 


Now I have a dictionary with each piece and how many times it occurs. Now I sort alphabetically. 
I look at all occurances starting with A and count. Then for each of A,T,G and C. I have the transition probabilites.


'''

dict_of_pieces = {}

for i in range(Number_of_windows):
    #print(i)
    piece_of_interest = ""
    for j in range(order_of_markov+1):
        #print(j)
        piece_of_interest += seq_of_interest[i+j]
    #print(piece_of_interest)
    if piece_of_interest in dict_of_pieces:
        dict_of_pieces[piece_of_interest] += 1
    else:
        dict_of_pieces[piece_of_interest] = 1

all_possible_from_states = set() # this is a set

for key in dict_of_pieces.keys():
    all_possible_from_states.add(key[0:(order_of_markov)])
    
all_possible_from_states = sorted(list(all_possible_from_states))

print(all_possible_from_states)
dict_of_pieces

['A', 'C', 'G', 'T']


{'GA': 46,
 'AC': 21,
 'CG': 38,
 'AA': 10,
 'AT': 22,
 'TC': 12,
 'GC': 5,
 'GT': 4,
 'TT': 12,
 'TA': 10,
 'AG': 13,
 'TG': 4,
 'CC': 2}

In [73]:
transition_matrix_from_data = np.zeros((len(all_possible_from_states),4))
transition_matrix_from_data

# Now I need to fill this. For each element in the set I need to sit and count the keys that start with it and count
# how many go to A, T, G, C each.

for index_of_from, string in enumerate(all_possible_from_states):
    for key in dict_of_pieces.keys():
        if string == key[0:order_of_markov]:
            for index_of_to, letter in enumerate(Alphabets_of_DNA_seq_list):
                if key[-1] == letter:
                    transition_matrix_from_data[index_of_from,index_of_to] = dict_of_pieces[key]
                    
            


transition_matrix_from_data 
                    
            


np.sum(transition_matrix_from_data)

199.0

In [77]:
row_total = np.sum(transition_matrix_from_data, axis = 1)
transition_matrix_from_data = transition_matrix_from_data / row_total.reshape((len(all_possible_from_states),1))

'''
c = 1.0

smoothed_tm = (transition_matrix_from_data + c/len(transition_matrix_from_data)) / (1 + c) 
what is this again?? check if valid

smoothed_tm

We may need pseudocounts but currently we do not really need it. It is doing better without it. 
I will add it in later if needed
'''


transition_matrix_from_data

array([[0.15151515, 0.31818182, 0.1969697 , 0.33333333],
       [0.        , 0.05      , 0.95      , 0.        ],
       [0.83636364, 0.09090909, 0.        , 0.07272727],
       [0.26315789, 0.31578947, 0.10526316, 0.31578947]])

In [64]:
transition_matrix


array([[0.25, 0.25, 0.25, 0.25],
       [0.05, 0.05, 0.8 , 0.1 ],
       [0.8 , 0.05, 0.05, 0.1 ],
       [0.25, 0.25, 0.25, 0.25]])

In [66]:
Alphabets_of_DNA_seq_list

['A', 'C', 'G', 'T']

In [75]:
all_possible_from_states

['A', 'C', 'G', 'T']